In [ ]:
import skel_features
from standard_transform import v1dd_ds
import pandas as pd
import numpy as np
from scipy import stats
import umap

skel_dir = '../skeletonization/skeletons/v1dd'
feature_dir = 'features/v1dd'

height_bnds = [-10, 900]

In [ ]:
# df1 = pd.read_feather('../skeletonization/data/v1dd/v1dd_single_soma.feather')
# df2 = pd.read_feather('../skeletonization/data/v1dd/v1dd_single_soma_r2.feather')
df3 = pd.read_feather('../skeletonization/data/v1dd/v1dd_single_soma_r3.feather')
df = df3
# df = pd.concat((df1, df2))

In [ ]:
success = skel_features.extraction.extract_features_mp(
    df['pt_root_id'],
    skel_dir,
    height_bnds,
    v1dd_ds,
    feature_dir,
)

In [ ]:
raw_df = skel_features.io_utils.load_features(df['pt_root_id'], feature_dir)

In [ ]:
(
    feat_df,
    feat_cols,
    syn_pca,
    br_svd,
    keep_depth,
    ego_pca,
    scalers,
) = skel_features.assembly.assemble_features_from_data(
    raw_df.dropna(), n_syn_comp=6, n_branch_comp=3, n_syn_ego=5
)

feat_df = feat_df.dropna()

In [ ]:
feat_df.drop(feat_df.query('max_density == 0').index, inplace=True)

In [ ]:
zero_df = feat_df.query('max_density==0')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from caveclient import CAVEclient
client = CAVEclient('v1dd')

ct_df = client.materialize.query_table('manual_central_types')

In [ ]:
cids = client.materialize.get_cell_ids(feat_df['root_id'])

In [ ]:
feat_df['nucleus_id'] = cids

In [ ]:
feat_cols_nospace = [
    "tip_len_dist_dendrite_p50",
    "tip_tort_dendrite_p50",
    "num_syn_dendrite",
    "num_syn_soma",
    "path_length_dendrite",
    "radial_extent_dendrite",
    "syn_dist_distribution_dendrite_p50",
    "syn_size_distribution_soma_p50",
    "syn_size_distribution_dendrite_p50",
    "syn_size_dendrite_cv",
    "syn_depth_extent",
    "max_density",
    "radius_dist",
    "branch_svd0",
    "branch_svd1",
    "branch_svd2",
    "ego_count_pca0",
    "ego_count_pca1",
    "ego_count_pca2",
    "ego_count_pca3",
    "ego_count_pca4",
]

In [ ]:
feat_cols

In [ ]:
dat = feat_df[feat_cols].values
datz = stats.zscore(dat)

res = umap.UMAP(metric="euclidean", min_dist=0.05, n_neighbors=20)
xx = res.fit_transform(datz)
feat_df["umap0"] = xx[:, 0]
feat_df["umap1"] = xx[:, 1]

In [ ]:
feat_df = feat_df.merge(
    ct_df[['pt_root_id', 'cell_type']].rename(columns={'pt_root_id':'root_id'}),
    how='left',
    on='root_id',
)

In [ ]:
feat_df.drop(feat_df.query('cell_type=="Non-neuronal"').index, inplace=True)

In [ ]:
import pickle
with open('../../../Projects/MinnieColumn/notebooks/paper_versions/v507/data/ctype_hues.pkl', 'br') as f:
    ctype_hues = pickle.load(f)

In [ ]:
feat_df.cell_type.isnull().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4), dpi=300)
feat_df.merge(
    ct_df[["pt_root_id", "cell_type"]].rename(columns={"pt_root_id": "root_id"}),
    how="left",
    on="root_id",
)

sns.scatterplot(
    x="umap0",
    y="umap1",
    data=feat_df,
    s=2,
    alpha=0.2,
    color='k',
)

sns.scatterplot(
    x="umap0",
    y="umap1",
    data=feat_df.query('cell_type != "Unsure I"'),
    s=10,
    hue="cell_type",
    palette={
        "PYC": [0.3, 0.3, 0.3],
        "BC": (0.0, 0.10980392156862745, 0.4980392156862745),
        "BPC": (0.07058823529411765, 0.44313725490196076, 0.10980392156862745),
        "MC": (0.5490196078431373, 0.03137254901960784, 0.0),
        "NGC": (0.7215686274509804, 0.5215686274509804, 0.0392156862745098),
    },
)

ax.legend().set_bbox_to_anchor((1,1))

In [ ]:
import phenograph

from sklearn import ensemble, model_selection

In [ ]:
from xgboost import XGBClassifier
from bidict import bidict

bst = XGBClassifier(n_estimators=10, max_depth=7, learning_rate=1, objective='binary:logistic')

In [ ]:
feat_df['is_exc'] = (feat_df['cell_type']=="PYC").astype(int)

In [ ]:
model_selection.cross_validate(
    bst,
    feat_df.dropna(subset='cell_type')[feat_cols_nospace],
    feat_df.dropna(subset='cell_type')['is_exc'],
    cv=5,
)

In [ ]:
bst.fit(
    feat_df.dropna(subset="cell_type")[feat_cols],
    feat_df.dropna(subset="cell_type")["is_exc"],
)
feat_df['is_exc_pred'] = bst.predict(feat_df[feat_cols])
feat_df['is_exc_pred'].replace({1: True, 0: False}, inplace=True)

In [ ]:
i_oids = feat_df.query('~is_exc_pred')['root_id'].values
raw_df_i = raw_df.query('root_id in @i_oids')
(
    feat_df_i,
    feat_cols_i,
    syn_pca_i,
    br_svd_i,
    keep_depth_i,
    ego_pca_i,
    model_dict_i,
) = skel_features.assembly.assemble_features_from_data(raw_df_i.dropna(), n_syn_comp=8, n_branch_comp=3, n_syn_ego=5)

idf = feat_df_i.dropna()

idf = idf.merge(
    ct_df[['pt_root_id', 'cell_type']].rename(columns={'pt_root_id':'root_id'}),
    how='left',
    on='root_id',
)

In [ ]:
bst = XGBClassifier(n_estimators=10, max_depth=7, learning_rate=1, objective='multi:softmax')

ct_map = bidict(
    {'BC': 0, 'MC': 1, 'NGC': 2, 'BPC': 3}
)

idftr = idf.dropna(subset='cell_type').query('cell_type!="Unsure I"')
idftr['cell_type_int'] = idftr['cell_type'].replace(ct_map)

model_selection.cross_validate(
    bst,
    idftr[feat_cols_i],
    idftr['cell_type_int'],
    cv=6,
)

In [ ]:
bst = XGBClassifier(n_estimators=10, max_depth=7, learning_rate=1, objective='multi:softmax')

bst.fit(
    idftr[feat_cols_i],
    idftr["cell_type_int"],
)
idf['ct_pred'] = bst.predict(idf[feat_cols_i])
idf['ct_pred'].replace(ct_map.inv, inplace=True)

In [ ]:
dat = idf[feat_cols_i].values
datz = stats.zscore(dat)

resi = umap.UMAP(metric="euclidean", min_dist=0.05, n_neighbors=30)
X = resi.fit_transform(datz)
idf["umap0"] = X[:, 0]
idf["umap1"] = X[:, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=150)
sns.scatterplot(
    x='umap0',
    y='umap1',
    color=(0.5, 0.5, 0.5),
    alpha=0.5,
    data=idf,
    s=10,
)
sns.scatterplot(
    x='umap0',
    y='umap1',
    hue='cell_type',
    palette='Set1',
    data=idf.dropna(subset='cell_type').query('cell_type!="Unsure I"'),
    s=10,
)
ax.set_aspect('equal')

In [ ]:
ct_hues = {ct: clr for ct, clr in zip(['BC', 'MC', 'BPC', 'NGC'], sns.color_palette('Dark2', n_colors=4))}

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=300)
sns.scatterplot(
    x='umap0',
    y='umap1',
    # color=(0.5, 0.5, 0.5),
    hue='ct_pred',
    palette=ct_hues,
    alpha=0.5,
    data=idf,
    s=10,
)
sns.scatterplot(
    x='umap0',
    y='umap1',
    hue='cell_type',
    palette=ct_hues,
    data=idf.dropna(subset='cell_type').query('cell_type!="Unsure I"'),
    s=10,
    edgecolor='k',
    legend=False,
)
ax.set_aspect('equal')
ax.legend().set_bbox_to_anchor((1,1))

In [ ]:
G, _, _ = phenograph.cluster(
    datz,
    clustering_algo="leiden",
    k=10,
    primary_metric="euclidean",
    resolution_parameter=1.2,
)
idf["clust"] = G

In [ ]:
idf.pivot_table(
    index='clust',
    columns='ct_pred',
    values='root_id',
    aggfunc='count',
    fill_value=0,
)

In [ ]:
from nglui import statebuilder

In [ ]:
idf_s = idf.merge(
    df[['pt_root_id', 'pt_position']],
    left_on='root_id',
    right_on='pt_root_id',
    how='left',
)

In [ ]:
idf_s.query('soma_depth < 110').groupby('ct_pred').count()

In [ ]:
idf = idf.merge(
    df[['pt_root_id', 'id']],
    left_on='root_id',
    right_on='pt_root_id',
    validate='1:1',
)

In [ ]:
# client.annotation.create_table(
#     table_name='cell_type_skel_features_v1',
#     schema_name='cell_type_reference',
#     description='Cell type predictions for single nucleus objects within 175 microns of the centerline and with nucleus volume greater than 218, based on dendritic skeleton features similar to those in Schneider-Mizell et al. 2023. Classification system has excitatory and inhibitory info, cell type is more detailed, with four inhibitory types (Soma-targeting ProxTC, Dendrite-targeting DistTC, Inhibitory-targeting InhTC, and Sparsely targeting SparTC) and excitatory neurons labeled by layer and projection class for layer five: IT, NP, ET.',
#     voxel_resolution=[1,1,1],
#     reference_table='nucleus_detection_v0',
#     track_target_id_updates=True,
# )

In [ ]:
stage = client.annotation.stage_annotations('cell_type_skel_features_v1')

In [ ]:
inh_replace = {'BC': 'ProxTC', 'MC': 'DistTC', 'BPC': 'InhTC', 'NGC': 'SparTC'}

In [ ]:
idf['classification'] = 'inhibitory'

In [ ]:
stage.add?

In [ ]:
for _, row in idf.iterrows():
    stage.add(
        cell_type = inh_replace.get(row['ct_pred']),
        classification_system='inhibitory',
        target_id=row['id'],
    )

In [ ]:
client.annotation.upload_staged_annotations(stage)

In [ ]:
img, seg = statebuilder.from_client(client)
annos = []
for ct, clr in zip(sorted(idf_s['ct_pred'].unique()), sns.palettes.color_palette('tab20', n_colors=len(np.unique(idf_s['ct_pred'])))):
    annos.append(
        statebuilder.AnnotationLayerConfig(
            ct,
            mapping_rules=statebuilder.PointMapper('pt_position', linked_segmentation_column='pt_root_id'),
            filter_query=f"ct_pred=='{ct}'",
            color=clr,
            data_resolution=[1,1,1],
            linked_segmentation_layer=seg.name,
        )
    )
sb = statebuilder.StateBuilder([img, seg]+annos, client=client)

In [ ]:
sb.render_state(
    idf_s.query('cell_type not in @ct_hues').sort_values(by='soma_depth'),
    return_as='html',
)

In [ ]:
idf.groupby('ct_pred').count()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=150)
sns.scatterplot(
    x='umap0',
    y='umap1',
    color=(0.5, 0.5, 0.5),
    alpha=0.5,
    data=idf,
    s=5,
)
sns.scatterplot(
    x='umap0',
    y='umap1',
    hue='clust',
    palette='tab20',
    data=idf,
    s=5,
)
ax.legend().set_bbox_to_anchor((1,1))
ax.set_aspect('equal')

In [ ]:
idf_s = idf.merge(
    df[['pt_root_id', 'pt_position']],
    left_on='root_id',
    right_on='pt_root_id',
    how='left',
)

In [ ]:
from xgboost import XGBClassifier
from bidict import bidict




In [ ]:
bst = XGBClassifier(n_estimators=10, max_depth=7, learning_rate=1, objective='multi:softmax')
idftr = idf.dropna(subset='cell_type').query('cell_type!="Unsure I"')
idftr['cell_type_int'] = idftr['cell_type'].replace(ct_map)
bst.fit(
    idftr[feat_cols_i],
    idftr['cell_type_int'],
)


In [ ]:
idf['ct_pred_xgb'] = bst.predict(idf[feat_cols_i])
idf['ct_pred_xgb'] = idf['ct_pred_xgb'].replace(ct_map.inv)

In [ ]:
idf_s = idf.merge(
    df[['pt_root_id', 'pt_position']],
    left_on='root_id',
    right_on='pt_root_id',
    how='left',
)

In [ ]:
img, seg = statebuilder.from_client(client)

annos = []
for ct, clr in zip(
    np.unique(idf_s["ct_pred_xgb"]),
    sns.palettes.color_palette("husl", n_colors=len(np.unique(idf_s["ct_pred_xgb"]))),
):
    annos.append(
        statebuilder.AnnotationLayerConfig(
            f"{ct}",
            mapping_rules=statebuilder.PointMapper(
                "pt_position", linked_segmentation_column="pt_root_id"
            ),
            filter_query=f"ct_pred_xgb=='{ct}'",
            color=clr,
            data_resolution=[1, 1, 1],
            linked_segmentation_layer=seg.name,
        )
    )
sb = statebuilder.StateBuilder([img, seg] + annos, client=client)

In [ ]:
sb.render_state(idf_s, return_as='html')

In [ ]:
feat_df.query('is_exc_pred')

Excitatory neurons

In [ ]:
# e_oids = feat_df.query('is_exc_pred')['root_id'].values
e_oids = e_oids[~np.isin(e_oids, bad_l4)]
raw_df_e = raw_df.query('root_id in @e_oids')
(
    feat_df_e,
    feat_cols,
    syn_pca,
    br_svd,
    keep_depth,
    ego_pca,
    model_dict_e,
) = skel_features.assembly.assemble_features_from_data(raw_df_e.dropna(), n_syn_comp=10, n_branch_comp=3, n_syn_ego=5)

edf = feat_df_e.dropna()

feat_cols.pop(16)

In [ ]:
dat = edf[feat_cols].values
datz = stats.zscore(dat)

rese = umap.UMAP(metric="euclidean", n_components=2, min_dist=0.05, n_neighbors=20)
X = rese.fit_transform(datz)
edf["umap0"] = X[:, 0]
edf["umap1"] = X[:, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=150)
sns.scatterplot(
    x='umap0',
    y='umap1',
    hue='soma_depth',
    palette='magma',
    data=edf,
    s=2,
    alpha=0.7,
)

In [ ]:
def relabel_sorted(df, label_column, sort_column, ascending=True):
    sorted_label = (
        df.groupby(label_column)
        .agg({sort_column: "mean"})
        .sort_values(by=sort_column, ascending=ascending)
        .index.values
    )
    relabel_index = {x: ii for ii, x in enumerate(sorted_label)}
    df[label_column] = df[label_column].apply(lambda x: relabel_index[x])

In [ ]:
import tqdm

In [ ]:
edf = edf.reset_index(drop=True)

In [ ]:
ntimes = 100
hold_out = 0.95

res_param = 2
kph = 15

dat = edf[feat_cols].values
datz_all = stats.zscore(dat)

df_groups = pd.DataFrame(index=edf.index.astype(int))
for ii in tqdm.tqdm(np.arange(ntimes)):
    subset_inds = np.sort(
        np.random.choice(
            np.arange(dat.shape[0]),
            size=np.round(hold_out * dat.shape[0]).astype(int),
            replace=False,
        )
    )
    
    datz = datz_all[subset_inds]

    labels, G, Q = phenograph.cluster(
        datz, clustering_algo="leiden", k=kph, resolution_parameter=res_param,
    )

    colname = f"r_k{kph}_{ii}"
    df_groups.loc[subset_inds, colname] = labels

# if save_plots:
# df_groups.to_pickle("df_groups_onehot_100_v5.pickle")

In [ ]:
def eq_nan_dist(u, v):
    no_nan = ~pd.isna(u + v)
    is_eq = np.equal(u[no_nan], v[no_nan])
    return is_eq.astype(int).sum()

from scipy import spatial

N_co = spatial.distance.pdist(df_groups, eq_nan_dist)
N_square = spatial.distance.squareform(N_co)

from sklearn import cluster, metrics

n_cl = []
sils = []
dbscores=[]
for k in np.arange(2,30):
    n_cl.append(k)
    model = cluster.AgglomerativeClustering(
        n_clusters=k,
        affinity="precomputed",
        linkage="complete",
        compute_distances=True,
        compute_full_tree=True,
    )
    l = model.fit_predict(-N_square)
    sils.append(metrics.silhouette_score(N_square, l))
    dbscores.append(metrics.davies_bouldin_score(N_square, l))

In [ ]:
n_cl = []
sils = []
dbscores=[]
for k in np.arange(20,40):
    n_cl.append(k)
    model = cluster.AgglomerativeClustering(
        n_clusters=k,
        metric="precomputed",
        linkage="complete",
        compute_distances=True,
        compute_full_tree=True,
    )
    l = model.fit_predict(-N_square)
    sils.append(metrics.silhouette_score(N_square, l))
    dbscores.append(metrics.davies_bouldin_score(N_square, l))

In [ ]:
n_cl[9]

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3), nrows=2, sharex=True)
rel_ind = 9
ax[0].plot(n_cl, dbscores, color='k')
ax[0].plot(n_cl[rel_ind], dbscores[rel_ind], 'ro')
ax[0].set_ylabel('Davies-Bouldin')

ax[1].plot(n_cl, sils, color='k')
ax[1].plot(n_cl[rel_ind], sils[rel_ind], 'ro')
ax[1].set_ylabel('Silhouette')
_ = ax[1].set_xticks(np.arange(0, 20, 5))

sns.despine(ax=ax[0])
sns.despine(ax=ax[1])

In [ ]:
model = cluster.AgglomerativeClustering(
    n_clusters=n_cl[rel_ind],
    metric="precomputed",
    linkage="complete",
    compute_distances=True,
    compute_full_tree=True,
)
l = model.fit_predict(-N_square)

In [ ]:
edf['clust'] = l
relabel_sorted(edf, "clust", "soma_depth")

In [ ]:
G, _, _ = phenograph.cluster(
    datz,
    clustering_algo="leiden",
    k=15,
    primary_metric="euclidean",
    resolution_parameter=2,
)
edf["clust"] = G


In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=300)
sns.scatterplot(
    x="umap0",
    y="umap1",
    hue="clust_str",
    palette="tab20",
    data=edf,
    alpha=0.4,
    s=3,
    ax=ax,
)
# sns.scatterplot(
#     x="umap0",
#     y="umap1",
#     data=edf.query('clust==18'),
#     color='k',
#     s=5,
#     ax=ax,
# )

ax.legend().set_bbox_to_anchor((1,1))

In [ ]:
layer_bnds  = [100, 270, 400, 550, 750]

fig, ax = plt.subplots(dpi=200, figsize=(6,3))
sns.stripplot(
    x='clust_str',
    y='soma_depth',
    hue='clust_str',
    palette='tab20',
    data=edf,
    s=2,
    alpha=0.1,
    ax=ax,
    legend=False,
)
ax.hlines(layer_bnds, *ax.get_xlim(), linestyle=':', color='k', linewidth=1, zorder=-20)
sns.despine(ax=ax)
ax.invert_yaxis()

In [ ]:
# bad_l4 = edf.query('clust in [8]')['root_id'].values

In [ ]:
val = 'num_syn_dendrite'

fig, ax = plt.subplots(dpi=200, figsize=(6,3))
sns.stripplot(
    x='clust',
    y=val,
    hue='clust',
    palette='tab20',
    data=edf,
    s=2,
    alpha=0.1,
    ax=ax,
    legend=False,
)

sns.pointplot(
    x='clust',
    y=val,
    color='k',
    estimator='median',
    data=edf,
    ax=ax,
    join=False,
)
# ax.hlines(layer_bnds, *ax.get_xlim(), linestyle=':', color='k', linewidth=1, zorder=-20)
# ax.invert_yaxis()
sns.despine(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(3,3), dpi=300)
sns.scatterplot(
    x="umap0",
    y="umap1",
    hue="num_syn_dendrite",
    hue_norm=(1000, 15000),
    palette="magma",
    data=edf,
    s=1,
    legend=False
)

In [ ]:
edf['num_syn_total']  = edf['num_syn_dendrite'] + edf['num_syn_soma']

fig, ax = plt.subplots(figsize=(3, 6), dpi=300)
sns.scatterplot(
    x='syn_depth_dist_p5',
    y='soma_depth',
    hue='clust_str',
    palette='tab20',
    data=edf,
    s=1,
)
ax.vlines(layer_bnds, *ax.get_ylim(), linestyle=':', color='k', linewidth=1, alpha=0.2, zorder=-20)

ax.hlines(layer_bnds, *ax.get_xlim(), linestyle=':', color='k', linewidth=1, zorder=-20)

ax.invert_yaxis()
ax.set_aspect('equal')
ax.legend().set_bbox_to_anchor((1,1))

In [ ]:
edf['num_syn_total']  = edf['num_syn_dendrite'] + edf['num_syn_soma']

fig, ax = plt.subplots(figsize=(3, 6), dpi=300)
sns.scatterplot(
    x='syn_depth_dist_p95',
    y='soma_depth',
    hue='clust_str',
    palette='tab20',
    data=edf,
    s=2,
    alpha=0.1,
)

sns.scatterplot(
    x='syn_depth_dist_p95',
    y='soma_depth',
    hue='clust_str',
    palette='tab20',
    data=edf.query('clust_str in ["L4a","L4b","L4c","L4d"] or clust_str in ["L6b", "L6g"]'),
    s=2,
    edgecolor='k',
    linewidth=0.05,
    legend=None,
)

# ax.set_xlim(0,3)
ax.hlines(layer_bnds, *ax.get_xlim(), linestyle=':', color='k', linewidth=1, zorder=-20)

ax.invert_yaxis()
# ax.set_aspect('equal')
ax.legend().set_bbox_to_anchor((1,1))

In [ ]:
edf.query('soma_depth>300 and soma_depth<400 and max_density>1').root_id.values

In [ ]:
val = 'syn_depth_dist_p95'

fig, ax = plt.subplots(dpi=200, figsize=(6,3))
sns.stripplot(
    x='clust_str',
    y=val,
    hue='clust',
    palette='tab20',
    data=edf,
    s=2,
    alpha=0.1,
    ax=ax,
    legend=False,
)

sns.pointplot(
    x='clust_str',
    y=val,
    color='k',
    estimator='median',
    data=edf,
    ax=ax,
    join=False,
)
# ax.hlines(layer_bnds, *ax.get_xlim(), linestyle=':', color='k', linewidth=1, zorder=-20)
# ax.invert_yaxis()
ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='center')

sns.despine(ax=ax)

In [ ]:
pt_subtype = {
    'L2': [0],
    'L3': [1,2,3,4,5],
    'L4': [6,7,8,9,10],
    'L5': [11,15,17],
    'L5ET': [12,13,14],
    'L5NP': [16],
    'L6': [18,19,20,21,22,23,24,25,26,27,28],
}

In [ ]:
names_ord = list(name_rep.values())
edf['clust_str'] = edf['clust_str'].astype(
    pd.CategoricalDtype(
        names_ord,
        ordered=True
    )
)

In [ ]:
import string

In [ ]:
name_rep = {}
for k, vl in pt_subtype.items():
    if len(vl)>1:
        for jth, jj in enumerate(vl):
            name_rep[jj] = f"{k}{string.ascii_lowercase[jth]}"
            ii = ii+1
    else:
        name_rep[vl[0]] = f"{k}"

In [ ]:
edf['clust_str'] = edf['clust'].replace(name_rep)

In [ ]:
edf.query('clust_str.isna()')

In [ ]:
edf['valence'] = 'excitatory'

In [ ]:
client.annotation.get_tables()

In [ ]:
desc = """
Clustering neurons based on dendritic features, without proofreading.
Consensus clustering based on phenograph with resolution parameter = 2, leiden algorithm, k_neighbors = 15.
Consensus of 100 runs with 5% dropout, 29 categories based on silhouette and Davies–Bouldin index.
Excitatory names are based on depth-ordering within layer, with some continuity choices to define layer group.
Good enough for exploratory analysis, probably, but not publication worthy.
By Casey Schneider-Mizell.
"""

In [ ]:
client.annotation.get_table_metadata('nucleus_detection_v0')

In [ ]:
client.annotation.create_table(
    'cell_type_skel_features_subtype_experimental',
    'cell_type_reference',
    description=desc,
    voxel_resolution=[9,9,45],
    reference_table='nucleus_detection_v0',
    track_target_id_updates=True,
)

In [ ]:
stage = client.annotation.stage_annotations('cell_type_skel_features_subtype_experimental')

In [ ]:
for _, row in edf.iterrows():
    stage.add(target_id=row['cell_id'], classification_system='excitatory', cell_type=row['clust_str'])

In [ ]:
client.annotation.get_tables()

In [ ]:
client.annotation.get_annotation_count(
    'cell_type_skel_features_subtype_experimental'
)

In [ ]:
edf.head()

In [ ]:
client.materialize.version

In [ ]:
edf

In [ ]:
edf.to_feather('v1dd_cell_subclasses_experimental_v410.feather')

In [ ]:
edf['root_id'].values[47:50]

In [ ]:
is_latest = client.chunkedgraph.is_latest_roots(edf['root_id'].values)

In [ ]:
edf[~is_latest]

In [ ]:
import tqdm

In [ ]:
suggest_latest = {}
for root_id in tqdm.tqdm(edf[~is_latest].root_id.values):
    suggest_latest[root_id] = client.chunkedgraph.suggest_latest_roots(root_id)

In [ ]:
edf['root_id_curr'] = edf['root_id'].replace(suggest_latest)

In [ ]:
cell_ids = client.materialize.get_cell_ids(edf['root_id_curr'], timestamp='now')

In [ ]:
edf['cell_id'] = cell_ids

In [ ]:
edf.to_feather(

In [ ]:
idf.rename(columns=

In [ ]:
edf.query('clust==17').root_id.sample(10).values

In [ ]:
# pt_subtype = {
#     'L2': [0],
#     'L3': [1,2,3,4,5],
#     'L4': [6,7,8],
#     'L5': [9,11,15],
#     'L5ET': [10,12,14],
#     'L5NP': [13],
#     'L6': [16,17,18,19,20,21,22,23,24,25],
# }

In [ ]:
pt_rep = {}
for k,v in pt_subtype.items():
    for ct in v:
        pt_rep[ct] = k

In [ ]:
edf_s['ct_category'] = edf_s['clust'].replace(pt_rep)

In [ ]:
for _, row in edf_s.iterrows():
    stage.add(
        cell_type=row['ct_category'],
        classification_system='excitatory',
        target_id=row['id'],
    )

In [ ]:
client.annotation.get_table_metadata('cell_type_skel_features_v1')

In [ ]:
client.annotation.get_annotation_count('cell_type_skel_features_v1')

In [ ]:
pt_subtype = {
    0: 'L2IT',
    1: 'L3IT',
    2: 'L3IT',
    3: 'L3IT',
    4: 'L4IT',
    5: 'L4IT',
    6: 'L4IT',
    7: 'L4IT',
    8: 'L5IT',
    9: 'L5ET',
    10: 'L5ET',
    11: 'L5IT',
    12: 'L5NP',
}

In [ ]:
feat_df

In [ ]:
pf_df = client.materialize.tables.ariadne_axon_task(cell_type=['clean', 'complete', 'submitted']).query()

In [ ]:
pf_df['proofread'] = True

In [ ]:
edf_pf = edf.merge(
    pf_df[['pt_root_id', 'proofread']].rename(columns={'pt_root_id': 'root_id'}),
    on='root_id',
    how='left',
)

edf_pf['proofread'] = edf_pf['proofread'].fillna(False)

In [ ]:
edf_pf.query('proofread').groupby('clust').count()

In [ ]:
val = 'syn_depth_dist_p5'

fig, ax = plt.subplots(dpi=200, figsize=(6,3))
sns.stripplot(
    x='clust',
    y=val,
    hue='clust',
    palette='tab20',
    data=edf,
    s=2,
    alpha=0.1,
    ax=ax,
    legend=False,
)

sns.pointplot(
    x='clust',
    y=val,
    color='k',
    estimator='median',
    data=edf,
    ax=ax,
    join=False,
)
ax.hlines(layer_bnds, *ax.get_xlim(), linestyle=':', color='k', linewidth=1, zorder=-20)
ax.invert_yaxis()
sns.despine(ax=ax)

In [ ]:
label_col = "clust"
feat_cols_with_depth = ["soma_depth"] + feat_cols

vline_locs = np.cumsum(edf.groupby(label_col).count()["root_id"].values)
datz = stats.zscore(edf[feat_cols_with_depth].values)
h = datz.shape[1]

fig, ax = plt.subplots(figsize=(20, 4), dpi=300)
sns.heatmap(
    datz[edf.reset_index().sort_values(by=[label_col, "soma_depth"]).index].T,
    cmap="RdBu_r",
    center=0,
    vmin=-2,
    vmax=2,
)

ax.set_yticks(np.arange(datz.shape[1]) + 0.5)
ax.vlines(vline_locs, 0, h, color="k", linewidth=1, alpha=1)
_ = ax.set_yticks(np.arange(0, len(feat_cols_with_depth))+0.5)
_ = ax.set_yticklabels(feat_cols_with_depth, rotation=0, fontdict={"size": 5})
_ = ax.set_xticks(
    np.diff(np.concatenate(([0], vline_locs))) / 2
    + np.concatenate(([0], vline_locs[:-1]))
)
_ = ax.set_xticklabels(sorted(list(edf[label_col].unique())), rotation=0)

In [ ]:
edf.query('clust==14')

In [ ]:
edf_s = edf.merge(
    df[['pt_root_id', 'id', 'pt_position']],
    left_on='root_id',
    right_on='pt_root_id',
    how='left',
)

In [ ]:
edf_s = edf_pf.merge(
    df[['pt_root_id', 'pt_position']],
    left_on='root_id',
    right_on='pt_root_id',
    how='left',
)

In [ ]:
edf_pf.to_feather('../feature_extraction/excitatory_feature_d150.feather')

In [ ]:
statebuilder.AnnotationLayerConfig?

In [ ]:
from nglui import statebuilder

img, seg = statebuilder.from_client(client)
annos = []
clusts = np.unique(edf['clust'])
for ct, clr in zip(clusts, sns.palettes.color_palette('tab20', n_colors=len(np.unique(edf['clust'])))):
    annos.append(
        statebuilder.AnnotationLayerConfig(
            f"clust{ct}",
            mapping_rules=statebuilder.PointMapper('pt_position', linked_segmentation_column='pt_root_id'),
            filter_query=f"clust=={ct}",
            color=clr,
            data_resolution=[1,1,1],
            linked_segmentation_layer=seg.name,
            filter_by_segmentation=True
        )
    )
sb = statebuilder.StateBuilder([img, seg]+annos, client=client, view_kws={'background_color': [0,0,0]})

sb.render_state(
    edf_s.sort_values(by='soma_depth'),
    return_as='html',
)

In [ ]:
feat_df.query('root_id == 864691132655358167')

In [ ]:
edf.query('is_exc_pred').sort_values(by='num_syn_dendrite', ascending=False).head(20)

In [ ]:
feat_cols

In [ ]:
raw_df.query('root_id == 864691132958276967')

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(
    x='radius_dist',
    y='soma_depth',
    hue='area_factor',
    palette='magma',
    # hue_norm=(150,250),
    data=edf.query('clust==0'),
    ax=ax,
    edgecolor='k',
)
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots()
sns.histplot(
    x='radius_dist',
    data=edf.query('clust==0'),
    bins=50,
)

In [ ]:
feat_df.pivot_table(
    index='pheno',
    columns='cell_type',
    values = 'soma_depth',
    fill_value=0,
    aggfunc='count',
)

In [ ]:
dfct_e = feat_df.query('cell_type == "PYC"').reset_index(drop=True)

dat = dfct_e[feat_cols].values
datz = stats.zscore(dat)
res = umap.UMAP(metric="euclidean", n_neighbors=10)
xx = res.fit_transform(datz)
dfct_e["umap0"] = xx[:, 0]
dfct_e["umap1"] = xx[:, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=300)
sns.scatterplot(
    x='umap0',
    y='umap1',
    data=dfct_e,
    s=5,
    hue='soma_depth',
    hue_norm=(0,700),
    palette='viridis',
)
ax.vlines(13,0,10)
ax.hlines(2.5,0,18)

In [ ]:
dfct_e.shape

In [ ]:
dfct_e.query('umap0>13 and umap1>2.5').root_id.values